In [5]:
from tqdm import tqdm, trange
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import docx
import warnings
import re
warnings.filterwarnings('ignore')

## Предпросмотр данных

In [108]:
interesting_features = ['Мгновенная механическая скорость бурения ', 'Частота вращения шпинделя', 'Индекс углубки', 
                       'Глубина скважины (м)', 'Крутящий момент на шпинделе', 'Давление промывочной жидкости', 
                       'Расход топлива мгновенный в час по шине CAN', 'Расход промывочной жидкости', 'Осевая нагрузка']

In [109]:
check = pd.read_csv('Result_23.csv')

In [118]:
dt = pd.DataFrame()
for feature in tqdm(interesting_features, desc = 'Обработка признаков'):
    dt = dt.append(check[check['DESCRIPTION'] == feature][['VALUE', 'TIMESTAMP', 'DESCRIPTION']])
dt = dt.drop_duplicates()
# dt.to_csv('first_preprocessing.csv', encoding = 'utf-32')
df = dt.pivot_table(index = 'TIMESTAMP', columns = ['DESCRIPTION'], values = ['VALUE'])
df.reset_index(drop = False, inplace = True)
df['TIMESTAMP'] = df['TIMESTAMP'].replace(to_replace = '\.\d\d\d', value = '', regex = True)
timestamp = pd.to_datetime(df['TIMESTAMP'], format = '%Y-%m-%d %H:%M:%S')
df['TIMESTAMP'] = timestamp

Обработка признаков: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  4.76it/s]


In [206]:
df.columns.droplevel(0)

TypeError: droplevel() got an unexpected keyword argument 'inplace'

In [119]:
print(df.isna().sum().sum())

7015


In [129]:
total_sec = 60
new = pd.DataFrame()
for i in trange(df.shape[0], desc = 'Обработка пропусков'):
    if i == 0:
        if (df['TIMESTAMP'].iloc[i + 1] - df['TIMESTAMP'].iloc[i]).total_seconds() < total_sec:
            df.iloc[i: i + 2].fillna(method = 'bfill', inplace = True)
    elif i == df.shape[0] - 1:
        if (df['TIMESTAMP'].iloc[i] - df['TIMESTAMP'].iloc[i - 1]).total_seconds() < total_sec:
            df.iloc[i-1:i+1].fillna(method = 'ffill', inplace = True)
    else:
        if (df['TIMESTAMP'].iloc[i] - df['TIMESTAMP'].iloc[i - 1]).total_seconds() < total_sec:
            df.iloc[i-1:i+1].fillna(method = 'ffill', inplace = True)
        if (df['TIMESTAMP'].iloc[i + 1] - df['TIMESTAMP'].iloc[i]).total_seconds() < total_sec:
            df.iloc[i: i + 2].fillna(method = 'bfill', inplace = True)

Обработка пропусков: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91567/91567 [08:38<00:00, 176.45it/s]


In [142]:
print(7015 - df.isna().sum().sum())

3546


In [146]:
df.dropna(inplace = True)

### Детекция на выбросы

In [165]:
interesting_features = list(df.columns)[1:14]

#### z-score

In [187]:
from scipy import stats
threshold = 2
dict_out_feat_z = {}
for feature in interesting_features:
    z = np.abs(stats.zscore(df[feature]))
    dict_out_feat_z[feature] = set(np.where(z > 2)[0])
set_all = set()
for feature in interesting_features:
    set_all = set_all|dict_out_feat_z[feature]b

#### IQR-test

In [242]:
np.where(df[feature] <= lower)

(array([], dtype=int64),)

In [244]:
dict_out_feat_iqr = {}
for feature in interesting_features:
    Q1 = np.percentile(df[feature], 25, interpolation='midpoint')
    Q3 = np.percentile(df[feature], 75, interpolation='midpoint')
    IQR = Q3 - Q1
    upper=Q3+1.5*IQR
    lower=Q1-1.5*IQR
    iqr = np.where((df[feature] < lower) | (df[feature] > upper))
    dict_out_feat_iqr[feature] = iqr

### Обучение алгоритма

#### Классический ML

In [395]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

In [396]:
target = [('VALUE','Частота вращения шпинделя')]
rmse = []
mse = []
r2 = []
mae = []
best_estimator = []
estimator = []
linear_models = [LinearRegression(), Lasso(), Ridge()]

In [397]:
X = df.drop(target + [df.columns[0]], axis = 1)
y = df[target]
parameters = {'alpha': np.arange(1, 100, 10)}

In [398]:
for linear_model in linear_models:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=True, random_state=123)
    linear = linear_model
    linear.fit(X_train, y_train)
    y_pred = linear.predict(X_test)
    mse.append(round(mean_squared_error(y_test, y_pred), 1))
    rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
    mae.append(round(mean_absolute_error(y_test, y_pred), 1))
    r2.append(round(r2_score(y_test, y_pred), 2))
    best_estimator.append('-')
    estimator.append(f'{str(linear_model).replace("()", "")}')
    if f'{linear_model}' == 'Ridge()' or f'{linear_model}' == 'Lasso()':
        linear= GridSearchCV(linear_model, parameters, scoring='neg_mean_squared_error',cv=5)
        linear.fit(X_train, y_train)
        y_pred = linear.predict(X_test)
        mse.append(round(mean_squared_error(y_test, y_pred), 1))
        rmse.append(round(mean_squared_error(y_test, y_pred) ** 0.5, 1))
        mae.append(round(mean_absolute_error(y_test, y_pred), 1))
        r2.append(round(r2_score(y_test, y_pred), 2))
        best_estimator.append(f'{linear.best_estimator_}')
        estimator.append(f'{str(linear_model).replace("()", "")}' + " opt")


In [399]:
dict_metrics = {'estimator': estimator,
              'mse': mse,
              'rmse': rmse,
              'r2': r2,
              'mae': mae,
              'best_estimator': best_estimator}
metrics = pd.DataFrame(dict_metrics)

## Создание документа

In [402]:
doc = docx.Document()
# задаем стиль текста по умолчанию
style = doc.styles['Normal']
# название шрифта
style.font.name = 'Times New Roman'
# размер шрифта
style.font.size = docx.shared.Pt(14)

In [403]:
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
line_spacing = 1.75

# заголовок 1
head_1 = doc.add_heading('Запрос в SQL', level = 1)
head_1.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_1.paragraph_format
h_fmt.line_spacing = 1.75
# параграф 
paragraph = doc.add_paragraph('SELECT df.ID, TIMESTAMP, dj.DESCRIPTION, VALUE  FROM DRILL_GEO_GAUGE_TRANS df \n\
LEFT JOIN DRILL_GEO_GAUGE_TYPE dj ON dj.ID = df.GAUGE_TYPE_ID \n\
ORDER BY df.ID;')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75
# заголовок 2
head_2 = doc.add_heading('Первичный осмотр данных', level = 1)
head_2.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_2.paragraph_format
h_fmt.line_spacing = 1.75
# параграф
paragraph = doc.add_paragraph(f'Всего имеется {check.shape[0]} наблюдений.\n\
Имееются следующие переменные: ')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.5
for var in check['DESCRIPTION'].unique():
    paragraph = doc.add_paragraph(f'{var}', style= 'List Number')
    p_fmt = paragraph.paragraph_format
    p_fmt.line_spacing = 1.75
# табличка
miss = check.isna().sum().reset_index()
table = doc.add_table(rows=check.shape[1] + 1, cols=2, style='Table Grid')
cell = table.cell(0, 0)
cell.paragraphs[0].add_run('Название переменной')
cell = table.cell(0, 1)
cell.paragraphs[0].add_run('Количество пропусков')

for column, value, index in zip(miss['index'], miss[0], range(1, miss.shape[0] + 1)):
    cell = table.cell(index, 0)
    cell.paragraphs[0].add_run(column)
    cell = table.cell(index, 1)
    cell.paragraphs[0].add_run(f'{value}')
# параграф про дубликаты
paragraph = doc.add_paragraph(f'Количество дубликатов в данных: \
{df.shape[0] - df.drop_duplicates().shape[0]} строчки')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75

# подглава про выбросы
head_3 = doc.add_heading('Детекция выбросов', level = 2)
head_2.alignment = WD_ALIGN_PARAGRAPH.CENTER
h_fmt = head_2.paragraph_format
h_fmt.line_spacing = 1.75

# текст с табличкой 
paragparh = doc.add_paragraph('В таблице предоставленны данные о выбросах')
p_fmt = paragraph.paragraph_format
p_fmt.line_spacing = 1.75
table = doc.add_table(rows = 5, cols = 9, style = 'Table Grid')
cell = table.cell(0, 0)
cell.paragraphs[0].add_run('Название метода\Название переменной')
cell = table.cell(1, 0)
cell.paragraphs[0].add_run('z-score (абсолютный)')
cell = table.cell(2, 0)
cell.paragraphs[0].add_run('z-score (в процентах)')
cell = table.cell(3, 0)
cell.paragraphs[0].add_run('IQR (абсолютный)')
cell = table.cell(4, 0)
cell.paragraphs[0].add_run('IQR (в процентах)')
for i in range(0, 8):
    cell = table.cell(0, i + 1)
    cell.paragraphs[0].add_run(df.columns.droplevel(0)[i + 1])
    cell = table.cell(1, i + 1)
    cell.paragraphs[0].add_run(f'{len(dict_out_feat_z[df.columns[i + 1]])}')
    cell = table.cell(2, i + 1)
    cell.paragraphs[0].add_run(f'{(len(dict_out_feat_z[df.columns[i + 1]]) / df.shape[0]) * 100 :.1f} %')
    cell = table.cell(3, i + 1)
    cell.paragraphs[0].add_run(f'{len(dict_out_feat_iqr[df.columns[i + 1]][0])}')
    cell = table.cell(4, i + 1)
    cell.paragraphs[0].add_run(f'{(len(dict_out_feat_iqr[df.columns[i + 1]][0]) / df.shape[0]) * 100:.1f} %')

# новая глава: машинное обучение
head_4 = doc.add_heading('Результаты машинного обучения', level = 1)
table = doc.add_table(metrics.shape[0]+1, metrics.shape[1])
for j in range(metrics.shape[-1]):
    table.cell(0,j).text = metrics.columns[j]
for i in range(metrics.shape[0]):
    for j in range(metrics.shape[-1]):
        table.cell(i+1,j).text = str(metrics.values[i,j])
# сохранение 
doc.save('Первичный осмотр.docx')